In [231]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from collections import defaultdict

plt.rcParams["figure.figsize"] = (15,8) #set size of plot

# Start working

## Finding calories needed for each country
The first need we need to do for our analysis is to find the total calories demand in each country... **write other things to start**

### Working on calories demand  
We load the calories demand datasets we scraped for the webpage [Calories](https://health.gov/dietaryguidelines/2015/guidelines/appendix-2/), datasets will we working on to match with population data

In [232]:
male_calory_demand = pd.read_excel("data/calories_demand.xlsx",header =None, sheet_name=0, names=['age', 'sedentary', 'moderate', 'active'])

In [233]:
females_calory_demand =  pd.read_excel("data/calories_demand.xlsx",header =None, sheet_name=1, names=['age', 'sedentary', 'moderate', 'active'])

In order to better work with the information we have collected, we will make some simplifications on the data. Mainly, we will:
- in the calories demands database, assume an average of necessary input per age
- group the ages into ranges that match the ranges provided in the World Population Database

In [234]:
def input_average(data_frame):
    result = data_frame.copy()
    result['input (KCal)'] = result.mean(axis=1) #computing the mean
    result = result.drop(columns=['sedentary', 'moderate', 'active']) #we keep only the mean
    return result

In [235]:
male_calories_avg = input_average(male_calory_demand)
females_calories_avg = input_average(females_calory_demand)

We have now obtained a caloric demand average for simpler calculations in the future and stored in the two precedent datasets.  
Now, we need a  way to match the age groups in this dataframe, to the ones in the population database we obtained. As such, let's analyse how ages are represented in our calory demand dataframes.

In [236]:
male_calories_avg['age'].unique()

array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       '19-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50',
       '51-55', '56-60', '61-65', '66-70', '71-75', '76 and up', nan],
      dtype=object)

We can see there are ranges of ages with different sizes (which makes sense, because different age groups have different caloric needs). We'll present a function that creates one row per individual age

In [237]:
def single_age(age_range):
    if type(age_range) ==  float: # nans are the only floats in the age column
        return -1
    elif type(age_range) == int:
        return age_range
    elif re.search('\d-\d', age_range):
        group = age_range.split('-')
        return list(range(int(group[0]), int(group[1])+1))
    elif age_range == "76 and up":
        return list(range(76, 101+1))

In [238]:
def explode_age(data_frame):
    accum = []
    for i in data_frame.index:
        row = data_frame.loc[i]
        single = single_age(row['age'])
        if single == -1: # we ignore the nan values, as their rows are empty
            continue
        if type(single) == int:
            accum.append((single, row['input (KCal)']))
        elif type(single) == list:
            accum.extend([(x, row['input (KCal)']) for x in single]) 
    return pd.DataFrame(accum, columns=data_frame.columns)

We apply the function to our two dataframe:

In [239]:
male_explode = explode_age(male_calories_avg)
female_explode = explode_age(females_calories_avg)

In [240]:
male_explode['age'].unique()

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101])

Ages are now unique in each dataframe ( `male_explode` and `female_explode` ) and there's a caloric input value for each of them.

#### Joao please comment the next two function and the code in the cells

The last step to allow the match with the population database is to build the **same age groups** we have in that set.  
We do this in the next two functions:

In [241]:
def group(age):
    i = int(5*(age//5))
    return "{}-{}".format(i, i+4)

In [242]:
def compress_ages(data_frame):
    accum = defaultdict(list)
    for i in data_frame.index:
        row = data_frame.loc[i]
        g_id = group(row['age'])
        if g_id == "100-104":
            g_id = "100+"
        accum[g_id].append(row['input (KCal)'])
    for i in accum:
        accum[i] = sum(accum[i]) / len(accum[i])
    return pd.DataFrame.from_dict(accum, orient='index')

We can lastly apply the functions to the dataframes:

In [243]:
new_male_need = compress_ages(male_explode)
new_female_need = compress_ages(female_explode)

We also use the age group as new index and rename the columns:

In [244]:
new_male_need.index.name = 'age_group'
new_male_need = new_male_need.rename(columns={0: 'input (KCal)'})
new_female_need.index.name = 'age_group'
new_female_need = new_female_need.rename(columns={0: 'input (KCal)'})

Let's see the result we have achieved and collected in our matchable dataframe `new_male_need` and `new_female_need`

In [245]:
new_male_need.head()

,input (KCal)
age_group,
0-4,1222.222222
5-9,1613.333333
10-14,2133.333333
15-19,2760.000000
20-24,2746.666667


## Computing total calories by matching 

**We now move on the matching with the *World Population Database* (United Nation) to compute the total calories needed in each country since 1950 to 2020**  
Matching with a precise dataset like this one is essential to take count of the demographics inside each country  
  
  
Firstly, we load the list of African countries (to filter the DB) and the two Databases (one for males, the other for females)  
*A note*: the values in the population dataframe are reported in **thousands**

In [382]:
with open ("data/african_countries.txt",'r', encoding='latin-1') as af_c:
    af_countries = [line.rstrip() for line in af_c] #loading list

In the next cell we are going to remove a list of countries for which **no data is availble** in the FAO Database that we will analyse later on in the notebook. The countries that will be deleted are the following: **Burundi, Comoros, Democratic Republic of the Congo, Equatorial Guinea, Libya ,Seychelles, Western Sahara, South Sudan, and Somalia**. Furthermore we noticed that in this list there are two French islands which are indipendent and hence beyond our analysis. As a consequence, we decide to drop these two small island **Mayotte** and **Réunion**.

In [399]:
af_to_remove = ['Burundi','Comoros','Democratic Republic of the Congo','Equatorial Guinea','South Sudan','Libya','Seychelles','Western Sahara','Somalia','Mayotte','Réunion']
af_countries = [i for i in af_countries if not i in af_to_remove]

Loading and cleaning the dataset to be prepared for merging (they represent unuseful columns with long names)

In [385]:
#loading datasets
pop_male = pd.read_excel("data/POPULATION_BY_AGE_MALE.xlsx", sheet_name="ESTIMATES")
pop_female = pd.read_excel("data/POPULATION_BY_AGE_FEMALE.xlsx", sheet_name="ESTIMATES")

In [386]:
#cleaning male population dataset
pop_male.drop(columns=["Index", "Variant", "Notes", "Country code", "Type", "Parent code"], inplace=True)
pop_male.rename(columns={"Reference date (as of 1 July)": "year", "Region, subregion, country or area *": "country"}, inplace=True)
#taking only african countries
pop_male = pop_male[pop_male['country'].isin(af_countries)]

#cleaning female dataset
pop_female.drop(columns=["Index", "Variant", "Notes", "Country code", "Type", "Parent code"], inplace=True)
pop_female.rename(columns={"Reference date (as of 1 July)": "year", "Region, subregion, country or area *": "country"}, inplace=True)
#only african
pop_female = pop_female[pop_female['country'].isin(af_countries)]

Now we multiply each column of the population data (as we said, in thousand) for each matching `age_group` in the calories table (that here we squeeze to allow the multiplication, similar to a transpose rows/columns of the dataset).  
We obtain two datasets: `total_cal_male` and `total_cal_female` reporting total calories needed for **each country in each year per age group per gender**

In [387]:
#total calories male
pop_mal_mult = pop_male.drop(columns=["country", "year"])
male_mult_res = pop_mal_mult.multiply(new_male_need.squeeze()) # squeeze adapts the dimension of the dataframe
#rejoin with old dataframe and delete old column (just population)
total_cal_male = pop_male.join(male_mult_res, lsuffix="_old")
total_cal_male = total_cal_male[total_cal_male.columns[~total_cal_male.columns.str.endswith('_old')]]

In [388]:
#total calories female
pop_fem_mult = pop_female.drop(columns=["country", "year"])
female_mult_res = pop_fem_mult.multiply(new_female_need.squeeze())
total_cal_female = pop_female.join(female_mult_res, lsuffix="_old")
total_cal_female = total_cal_female[total_cal_female.columns[~total_cal_female.columns.str.endswith('_old')]]

Once we have the calories needed for both gender, we can add them together easily to achieve total calories needed for **each country in each year per age group**, and we collect them in the dataframe `total_cal_ages`

In [389]:
#copy the male inside the total (to mantain country and year columns) and sum with female
total_cal_ages = total_cal_male.copy()
sum_ind = total_cal_ages.columns[2:]
total_cal_ages[sum_ind] = total_cal_ages[sum_ind] + total_cal_female[sum_ind]

To find the total calories needed for **each country in each year** we then proceed to the sum over all the age groups, collected in `total_cal`

In [390]:
total_cal = total_cal_ages.copy()
sum_ind = total_cal.columns[2:]

#computing sum of cal over ages, removing ages
total_cal['Calories'] = total_cal[sum_ind].sum(axis=1)
total_cal.drop(columns=sum_ind, inplace=True)

The values proposed up to here **need to be scaled appropriartely**, as the total amount of calories should be multiplied by 1000 to take in account the value reported in the population dataset.  
Instead of dealing with large number (up to order of $10^9$ Kcal), we decide to divide another time by 1000.  
The reader is so advised that from now on all the calories will be reported in **Gcal** (I'm not totally sure, check together ahah) 

In [258]:
change_col = total_cal_ages.columns[2:] #index for the first 3 dataframes are the same
total_cal_male[change_col] = total_cal_male[change_col]/1000
total_cal_female[change_col] = total_cal_male[change_col]/1000
total_cal_ages[change_col] = total_cal_male[change_col]/1000
total_cal['Calories'] = total_cal['Calories']/1000

Drawing a sample of the final dataframe..

In [259]:
total_cal.head()

,country,year,Calories
420,Djibouti,1950,121.291427
421,Djibouti,1955,138.432262
422,Djibouti,1960,166.978938
423,Djibouti,1965,227.616138
424,Djibouti,1970,313.232320


#### Summing up to now
* We have defined a reasonable amount of calories needed for each gender and each group by taking average
* We collected these values in `new_male_need` and `new_female_need`
* We loaded the population of the African countries from the United Nation Dataset 
* We matched the population for the kcal needed by each age group. From this matching we build 4 different datasets, with different granularity levels:  
`total_cal_male`, `total_cal_female`, `total_cal_ages`, `total_cal`
* We changed the scale of our final dataframes to easily work with large numbers

We now proceed on by importing the **Food Balance Sheet** from FAO Dataset (African countries only). 

In [665]:
FAO_food_balance = pd.read_csv("Data/FoodBalanceSheets_E_Africa_1.csv",encoding='latin-1');
FAO_food_balance.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,4,Algeria,2501,Population,511,Total Population - Both sexes,1000 persons,11561.0,NaN,11845.0,...,36383.0,NaN,37063.0,NaN,37763.0,NaN,38482.0,NaN,39208.0,NaN
1,4,Algeria,2511,Wheat and products,5511,Production,1000 tonnes,686.0,S,1507.0,...,2953.0,S,2605.0,S,2555.0,S,3432.0,S,3299.0,S
2,4,Algeria,2511,Wheat and products,5611,Import Quantity,1000 tonnes,469.0,S,501.0,...,5757.0,S,5109.0,S,7487.0,S,6385.0,S,6343.0,S
3,4,Algeria,2511,Wheat and products,5072,Stock Variation,1000 tonnes,353.0,S,-409.0,...,-484.0,S,515.0,S,-1050.0,S,-350.0,S,-180.0,S
4,4,Algeria,2511,Wheat and products,5911,Export Quantity,1000 tonnes,46.0,S,12.0,...,37.0,S,39.0,S,5.0,S,5.0,S,2.0,S


Firstly, we will **remove** all the columns with title **"Y----F"** as they contain informations about how the data were obtained (Calculated, Regression, Aggregate, FAO Estimation). In this context we will consider that FAO is an highly renowned Agency and hence we can assume these values are truthful without loss of generality. Furthermore we thought that it would be very handy to have numbers as columns representing years instead of **"Y----"**. We proceed on removing the letter **Y**.

In [667]:
col_to_drop = FAO_food_balance.columns[FAO_food_balance.columns.str.endswith("F")]; #creating a list of columns to be removed
FAO_food_balance = FAO_food_balance.drop(columns=col_to_drop);
FAO_food_balance.columns=FAO_food_balance.columns.str.replace("Y","")

Secondly, we replace all the **NAN** values with **0** as Item was not available.

In [668]:
FAO_food_balance = FAO_food_balance.fillna(0);

The third step to complete **the cleaning** of FAO_food_balance consists into adapt names of countries in order to have consistency along our different dataframes. In particular we will have to match with our list of African countries **af_countries**. Moreover, we noticed that **Sudan** appears twice as "Sudan" and "Sudan (former)". This is due to the fact that South Sudan gained its independecy in 2011 (reference to https://en.wikipedia.org/wiki/South_Sudan). The data suggests that even after the , Sudan accounts for both countries in terms of food supply and for this reason we will just consider them as if it was one single country.

In [669]:
FAO_food_balance['Area'] = FAO_food_balance['Area'].replace("Swaziland", "Eswatini");

****---------------DELETING SUDAN FORMER AND MERGE TWO ROWS TO BE DONE HERE--------****

Analysing our DataFrame **FAO_food_balance** we can see that it's already well structured since it contains many key - value couples such as **Item Code - Item** and **Element Code - Element** . More specifically, we will take advantage of this structure to filter out only rows containing **Food supply (kcal/capita/day)** as an **Element**. The corresponding key to this value is **664**. 

In order to keep our original Dataframe **FAO_food_balance** as a reference we create a new Dataframe **FAO_food_supply** in which we keep just **countries** and **food supplies** for every **year**.

In [670]:
FAO_food_supply = FAO_food_balance[FAO_food_balance["Element Code"]==664];
FAO_food_supply = FAO_food_supply.drop(columns=["Area Code","Item Code","Item","Element Code","Element", "Unit"])
FAO_food_supply.head()

,Area,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
13,Algeria,806.0,843.0,777.0,763.0,822.0,844.0,889.0,890.0,883.0,...,1462.0,1458.0,1378.0,1358.0,1409.0,1452.0,1385.0,1420.0,1433.0,1424.0
26,Algeria,7.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,5.0,...,20.0,15.0,19.0,21.0,27.0,20.0,28.0,21.0,27.0,30.0
40,Algeria,160.0,139.0,177.0,162.0,171.0,139.0,144.0,180.0,185.0,...,106.0,104.0,123.0,130.0,98.0,98.0,97.0,96.0,96.0,94.0
53,Algeria,4.0,4.0,2.0,4.0,5.0,4.0,9.0,6.0,8.0,...,77.0,78.0,154.0,172.0,152.0,107.0,151.0,119.0,128.0,129.0
61,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can now group group by **Area** and sum supplies derived from each item available in countries for that particular year.

In [673]:
FAO_food_supply = FAO_food_supply.groupby("Area").sum();
FAO_food_supply.head()

,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Area,,,,,,,,,,,,,,,,,,,,,
Algeria,6471.0,6270.0,6104.0,6157.0,6362.0,6281.0,6582.0,6818.0,6817.0,6696.0,...,11943.0,11829.0,12185.0,12159.0,12184.0,12440.0,12566.0,12866.0,13088.0,13178.0
Angola,7190.0,7270.0,7409.0,7444.0,7502.0,7558.0,7679.0,7418.0,7785.0,7858.0,...,8122.0,8304.0,8471.0,8702.0,8983.0,9213.0,9375.0,9627.0,9531.0,9893.0
Benin,6947.0,7036.0,6811.0,6678.0,7248.0,7219.0,7333.0,7743.0,7486.0,7247.0,...,9841.0,9740.0,9799.0,10252.0,10084.0,10258.0,10216.0,10393.0,10440.0,10477.0
Botswana,7903.0,7640.0,7889.0,8000.0,8023.0,7787.0,8041.0,8205.0,8053.0,8141.0,...,8761.0,8792.0,8595.0,8663.0,8736.0,8789.0,8936.0,9081.0,9366.0,9303.0
Burkina Faso,5300.0,5230.0,5809.0,6220.0,6287.0,6241.0,6348.0,6405.0,6411.0,6181.0,...,10021.0,9854.0,10236.0,10180.0,10350.0,10554.0,10718.0,10653.0,10825.0,10878.0
